In [ ]:
import requests
import json
import sys
import os

## --- parameter ---
CONTEST = "W397" # Contest name
CONTEST_URL = "https://leetcode.com/contest/api/ranking/weekly-contest-397/" # API URL

# HTTP request header
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Referer": "https://leetcode.com/contest/",
    "Sec-Fetch-Site": "same-origin",
}

In [ ]:
# init: create folder and write the header of csv
try:
    os.mkdir(CONTEST)
    os.mkdir(CONTEST+"/Q3")
    os.mkdir(CONTEST+"/Q4")
    os.mkdir(CONTEST+"/Q3/0001-1000")
    os.mkdir(CONTEST+"/Q3/1001-2000")
    os.mkdir(CONTEST+"/Q3/2001-3000")
    os.mkdir(CONTEST+"/Q3/3001-4000")
    os.mkdir(CONTEST+"/Q3/4001-5000")
    os.mkdir(CONTEST+"/Q3/5001-6000")
    os.mkdir(CONTEST+"/Q4/0001-1000")
    os.mkdir(CONTEST+"/Q4/1001-2000")
    os.mkdir(CONTEST+"/Q4/2001-3000")
    os.mkdir(CONTEST+"/Q4/3001-4000")
    os.mkdir(CONTEST+"/Q4/4001-5000")
    os.mkdir(CONTEST+"/Q4/5001-6000")
except:
    pass

with open(CONTEST+"/"+CONTEST+".csv", "w") as f:
    f.write("user, Q3, Q4\n");

In [ ]:
## get question_id
for k in range(10):
    try:
        r = requests.get(CONTEST_URL,  headers=headers)
        response = json.loads(r.text)
    except:
        pass
    else:
        break
else:
    print("error")
    sys.exit()

for question in response["questions"]:
    print(question["question_id"], question["credit"], question["title"])

In [ ]:
def lang_ext(lang):
    match lang:
        case "cpp":
            return "cpp"
        case "python3" | "python":
            return "py"
        case "java":
            return "java"
        case _:
            return lang

## --- parameter ---
PAGE = 202 # how many page gonna fetch
Q = ["3391", "3431"] # [Q3, Q4], use question_id get from above

with open(CONTEST+"/"+CONTEST+".csv", "a") as f:
    for pagination in range(1, PAGE+1):
        for k in range(10):
            try:
                r = requests.get(CONTEST_URL+"?pagination=" + str(pagination) +"&region=global",  headers=headers)
                response = json.loads(r.text)
            except:
                pass
            else:
                break
        else:
            print(pagination)
            sys.exit()

        print(pagination)

        for i in range(25):
            f.write(response["total_rank"][i]["user_slug"])
            
            for j, Q_id in enumerate(Q):
                if Q_id not in response["submissions"][i]:
                    f.write(",")
                    continue
            
                submission_id = str(response["submissions"][i][Q_id]["submission_id"])
    
                f.write("," + submission_id)

                for k in range(10):
                    try:
                        r2 = requests.get("https://leetcode.com/api/submissions/" + submission_id,  headers=headers)
                        response2 = json.loads(r2.text)
                    except:
                        pass
                    else:
                        break
                else:
                    print(submission_id)
                    sys.exit()

                rank = (pagination-1)*25 + i + 1
                folder = str(rank//1000)+"001-"+str(rank//1000+1)+"000/"
                with open(CONTEST+"/Q"+str(j+3)+"/"+folder+str(rank)+"."+lang_ext(response2["lang"]), "w", encoding="utf-8") as f2:
                    try:
                        f2.write(response2["code"])
                    except:
                        print(submission_id)
                        sys.exit()

            f.write("\n")
    
print("finished!!!")